### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [102]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset - I changed the order of the data sets and made it a right join instead of left.
# That way it's a little easier for me to deal with later on. 
school_data_complete = pd.merge(school_data, student_data, how="right", on=["school_name", "school_name"])
school_data

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [103]:
#Calculate the total number of schools
# The value_counts method counts unique values in a column
school_count = school_data["school_name"].count()
student_count = student_data["student_name"].count()
total_budget = school_data["budget"].sum()
avg_read = student_data["reading_score"].mean()
avg_math = student_data["math_score"].mean()
overall_pass = (avg_read + avg_math)/2

#math
passed_math = [score for score in student_data["math_score"] if score >= 70]
pass_math = len(passed_math)
perc_pass_math = (pass_math/student_count) * 100
#perc_pass_math

#reading
passed_reading = [score for score in student_data["reading_score"] if score >= 70]
pass_reading = len(passed_reading)
perc_pass_read = (pass_reading/student_count) * 100
#perc_pass_read


district_summary_df = pd.DataFrame({"Total Schools": [school_count],
                                    "Total Students": [student_count],
                                    "Total Budget": [total_budget],
                                    "Average Math Score": [avg_math],
                                    "Average Reading Score": [avg_read],
                                    "% Passing Math": [perc_pass_math],
                                    "% Passing Reading": [perc_pass_read],
                                    "% Overall Passing Rate": [overall_pass]
                                   })

district_summary_df = district_summary_df.round(6)

district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{0:,.2f}%".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{0:,.2f}%".format)
district_summary_df["% Overall Passing Rate"] = district_summary_df["% Overall Passing Rate"].map("{0:,.2f}%".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${0:,.0f}".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428",78.985371,81.87784,74.98%,85.81%,80.43%


24649428

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [111]:
#Create data frame containing all necessary data for school summary questions
#Using "school_name", "type", "size", "budget", "gender", "reading_score", "math_score"


school_data_dist = pd.DataFrame(school_data_complete[["school_name", "type", "size", "budget", "gender", "reading_score", "math_score"]])
#school_data_dist = school_data_dist_raw.rename(columns={"school_name": "School", "type": "Type", "size": "Enrollment", "budget": "Budget", 
 #                                                       "gender": "Gender", "reading_score": "Reading Score", "math_score": "Math Score" })

#Gets the budgets of each school
total_budget_school = school_data_dist["budget"].unique()

#create a data frame grouped by schools to get enrollment
grouped_schools_df = school_data_dist.groupby(["school_name"])
enrollment = grouped_schools_df["size"].unique()

#calculates expenditure per student
budget_per_stu = total_budget_school/enrollment

#calculate the average math and reading scores per school
avg_math_school = grouped_schools_df["math_score"].mean()
avg_read_school = grouped_schools_df["reading_score"].mean()

#calculate number of students passing reading and math.
grouped_schools_math_df = school_data_dist.loc[school_data_dist["math_score"] >= 70].groupby(["school_name"])
grouped_schools_read_df = school_data_dist.loc[school_data_dist["reading_score"] >= 70].groupby(["school_name"])

pass_math = grouped_schools_math_df["math_score"].count()
pass_reading = grouped_schools_read_df["reading_score"].count()

#calculate percentage of students passing reading and math
perc_passing_math = (pass_math/enrollment) 
perc_passing_reading = (pass_reading/enrollment) 

#Overall passing rate per school
overall_pass_school = (perc_passing_reading + perc_passing_math)/2

school_avg_df_raw = pd.DataFrame({"Enrollment": enrollment,
                              "Average Math": avg_math_school,
                              "Average Reading": avg_read_school,
                              "Total Budget": total_budget_school,
                              "Budget Per Student": budget_per_stu,
                              "% Passing Math": perc_passing_math,
                              "% Passing Reading": perc_passing_reading,
                              "% Overall Passing": overall_pass_school})
                              

school_avg_df = school_avg_df_raw[["Enrollment",
                              "Average Math",
                              "Average Reading",
                              "Total Budget",
                              "Budget Per Student",
                              "% Passing Math",
                              "% Passing Reading",
                              "% Overall Passing"]]
                              
school_avg_df = school_avg_df.round()
school_avg_df

#print(f'Enrollment: {enrollment} students passing reading {pass_math} percentage passing reading {perc_passing_math}')
#print(f' overall pass: {overall_pass_school}')                              

,Enrollment,Average Math,Average Reading,Total Budget,Budget Per Student,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,
Bailey High School,[4976],77.0,81.0,1910635,[383.97005627009645],[0.6668006430868167],[0.8193327974276527],[0.7430667202572347]
Cabrera High School,[1858],83.0,84.0,1884411,[1014.2147470398278],[0.9413347685683531],[0.9703982777179763],[0.9558665231431647]
Figueroa High School,[2949],77.0,81.0,1056600,[358.29094608341813],[0.6598847066802306],[0.8073923363852153],[0.733638521532723]
Ford High School,[2739],77.0,81.0,3022020,[1103.329682365827],[0.6830960204454181],[0.7929901423877328],[0.7380430814165755]
Griffin High School,[1468],83.0,84.0,917500,[625.0],[0.9339237057220708],[0.9713896457765667],[0.9526566757493188]
Hernandez High School,[4635],77.0,81.0,1319574,[284.6977346278317],[0.667529665587918],[0.8086299892125135],[0.7380798274002158]
Holden High School,[427],84.0,84.0,1081356,[2532.4496487119436],[0.9250585480093677],[0.9625292740046838],[0.9437939110070257]
Huang High School,[2917],77.0,81.0,3124928,[1071.281453548166],[0.6568392183750429],[0.8131642098045938],[0.7350017140898184]
Johnson High School,[4761],77.0,81.0,248087,[52.10817055240496],[0.6605755093467759],[0.8122243226212981],[0.736399915984037]


In [112]:

school_avg_df = pd.DataFrame({#"School Type": school_type,
                              "Enrollment": enrollment,
                              "Average Math": avg_math_school,
                              "Average Reading": avg_read_school,
                              "Total Budget": total_budget_school,
                              "Budget Per Student": budget_per_stu,
                              "% Passing Math": perc_passing_math,
                              "% Passing Reading": perc_passing_reading,
                              "% Overall Passing": overall_pass_school})


#Format data in dataframe
school_avg_df = school_avg_df.round(6)

school_avg_df["% Passing Math"] = school_avg_df["% Passing Math"].map("{0:,.2f}%".format)
school_avg_df["% Passing Reading"] = school_avg_df["% Passing Reading"].map("{0:,.2f}%".format)
school_avg_df["% Overall Passing"] = school_avg_df["% Overall Passing"].map("{0:,.2f}%".format)
school_avg_df["Total Budget"] = school_avg_df["Total Budget"].map("${0:,.0f}".format)
school_avg_df["Budget Per Student"] = school_avg_df["Budget Per Student"].map("${0:,.0f}".format)

school_avg_df


TypeError: unsupported format string passed to numpy.ndarray.__format__

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [109]:
# Display top 5 performing schools in descending order
top_school_pass_df = school_avg_df.sort_values(["% Overall Passing"], ascending=False)
top_school_pass_df.head()





,Enrollment,Average Math,Average Reading,Total Budget,Budget Per Student,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,
Cabrera High School,[1858],83.061895,83.975780,1884411,[1014.2147470398278],[0.9413347685683531],[0.9703982777179763],[0.9558665231431647]
Thomas High School,[1635],83.418349,83.848930,3094650,[1892.7522935779816],[0.9327217125382263],[0.9730886850152906],[0.9529051987767585]
Pena High School,[962],83.839917,84.044699,585858,[609.0],[0.9459459459459459],[0.9594594594594594],[0.9527027027027026]
Griffin High School,[1468],83.351499,83.816757,917500,[625.0],[0.9339237057220708],[0.9713896457765667],[0.9526566757493188]
Wilson High School,[2283],83.274201,83.989488,1763916,[772.6307490144546],[0.938677179150241],[0.9653964082347788],[0.9520367936925098]


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [110]:
# Display bottom 5 performing schools in ascending order
top_school_pass_df = school_avg_df.sort_values(["% Overall Passing"], ascending=True)
top_school_pass_df.head()

,Enrollment,Average Math,Average Reading,Total Budget,Budget Per Student,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,
Rodriguez High School,[3999],76.842711,80.744686,1049400,[262.41560390097527],[0.6636659164791198],[0.8022005501375343],[0.732933233308327]
Figueroa High School,[2949],76.711767,81.158020,1056600,[358.29094608341813],[0.6598847066802306],[0.8073923363852153],[0.733638521532723]
Huang High School,[2917],76.629414,81.182722,3124928,[1071.281453548166],[0.6568392183750429],[0.8131642098045938],[0.7350017140898184]
Johnson High School,[4761],77.072464,80.966394,248087,[52.10817055240496],[0.6605755093467759],[0.8122243226212981],[0.736399915984037]
Ford High School,[2739],77.102592,80.746258,3022020,[1103.329682365827],[0.6830960204454181],[0.7929901423877328],[0.7380430814165755]


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [ ]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

## Scores by School Type

* Perform the same operations as above, based on school type.